# COGS 118A- Project Checkpoint

# Names

- Amy Hardy
- Dina Dehaini
- Stephanie Kwan
- Darren Wu
- Nicholas Hale

# Abstract 
Sentiment analysis can help business owners process huge amounts of feedback from reviewers and automatically tag them. In our project, we will implement sentiment analysis on over 500k Amazon reviews and attempt to accurately tag/predict reviews based on the actual text of the review. Labels will be derived from their ratings with ratings of 4-5 being positive, 3 neutral, and 1-2 negative. We will also attempt the regression task of predicting a rating between the continuous range of 1 and 5 stars using the review.  We will preprocess the text data through the typical means of removing stop words, stemming, lemmatization, tokenization, and vectorization. Next, an SVM, random forest classifier, and logistic regression model will be trained on the extracted features and evaluated on the remaining data with performance being measured through each model's accuracy in predicting both the labels and the ratings.

# Background


Sentiment Analysis is a powerful method for corporations and other organizations to analyze text for a variety of purposes. Sentiment analysis can be defined as the procedure of utilizing natural language processing, text analysis, and other metrics to determine the tonality of a body of text.<a name="one"></a>[<sup>[1]</sup>](#one) When utilized normally, the polarity of analyzed text is separated into categorizations, such as ‘Positive’, ‘Negative’, and ‘Neutral’<a name="four"></a>[<sup>[4]</sup>](#four).

The specific manners in which sentiment analysis can be utilized varies along a great range.	Sentiment analysis allows for a consistent metric to analyze text, and in the context of something such as product reviews, allows for the removal of unconscious bias in its analysis in addition to allowing for greater amounts of data to be processed. In a corporate context, sentiment analysis can be utilized to gain realtime insights into consumer opinions and satisfaction and allow for trends to be identified by a text’s polarity.

In the context of machine learning, there are a number of algorithms that can be used to analyze text, from Naive Bayes, Linear Regression, Support Vector Machines, neural networks, and more.<a name="two"></a>[<sup>[2]</sup>](#two)

Each model has challenges with their implementations, and there are general challenges with sentiment analysis itself in its current state. Text is very nuanced, and there are certain aspects of tone, sentence structure, slang, subjectivity, and more that cause for necessity of improvements upon current models to allow this tool to improve in accuracy.<a name="three"></a>[<sup>[3]</sup>](#three) With further research into this topic, it is possible to identify specific algorithms that allow for the most accuracy when performing sentiment analysis. Performing sentiment analysis on the same dataset multiple times while varying the models used will allow for one to identify which model provides the greatest accuracy for this specific type of data. Product reviews invite different language use by those submitting them than other forms of text, and as mentioned before, performing sentiment analysis on this form of data is largely beneficial to corporations and other parties that may be interested in consumer feedback.


# Problem Statement

The problem we will be tackling in this project is how to best classify reviews in terms of positive, neutral, and negative as well as in terms of rating. We want to compare how these two different kinds of outputs will perform, and to see which method will produce a better accuracy. We will implement an SVM, random forest classifier, and logistic regression based sentiment analysis model. Specifically, each model will take as input several training reviews and their ratings and then test its accuracy against several test reviews and their ratings by classifying the polarity of a given Amazon review and automatically tag it as positive, neutral, or negative, and also see if we can then give the review a rating. 

# Data

The dataset can be found at https://www.kaggle.com/datasets/jillanisofttech/amazon-product-reviews.
 
This dataset has a total of 568k customer reviews. Each review includes the following fields:
- ProductID: the id of the Amazon product
- UserID: the id of the Amazon user who is reviewing the product
- ProfileName: the profile name of the user who is reviewing the product
- HelpfulnessNumerator: number of people who found the review helpful
- HelpfulnessDenominator: number of people who did not find the review helpful
- Score: the rating the user is giving the product, on a scale of 1-5 stars
- Time: the timestamp that the review was posted at
- Summary: a short phrase that summarizes the review
- Text: the actual text of the review

For our project, we will be dropping most features besides the variables that contain text (customer reviews/summary) and rating. The UserID and ProfileName will provide no help in classifying a review’s sentiment, and neither will the timestamp. The values in the helpfulness numerator and denominator often match in each row, which makes their usage and meaning confusing, and overall we believe that the fields will not provide our model additional useful information. We plan to drop the summary field initially as well.

An additional column for classification will be made based on the score column –- numbers 1-2 will be -1 for “negative”, 3 will be 0 for “neutral”, and 4-5 will be 1 for “positive” in the new classification column. The reviews and summary will undergo text preprocessing and vectorization before their use in training the models.

## Cleaned Data
Due to the size of the original CSV file being ~300MB, we had to split it into 4 separate, smaller CSV files so that we could use GitHub. We then wrote a function to retrieve and merge these files together and return the completed dataset.

## EDA

Taking a closer look at the data, we can see it's heavily skewed and we have a disportionate number of positive reviews. This may impact our model's ability to accurately predict negative review or neutral reviews. This in turn could lead to a higher true positive rate but also a higher false positive rate meaning we'd have a more sensitive and specific model. 

We also used wordclouds to look at some of the more frequent words based on their sentiment label for both the "Review" and "Summary" columns. From these plots we can see another potential issue: our model may start to associate neutral words with positive or negative sentiments. Because the products these reviews come from have a star rating and many of the reviews come from these same products, we are left with many reviews for the same product in the same sentiment category. Of course many of these reviews also mention the name of the product (i.e. dog food) and so these words become the most frequent in this sentiment category thus creating an issue for us. Unfortunately, product names were not included as a feature so dropping them from the reviews or summaries may not be possible. Looking at the wordclouds for the summaries, we notice slightly better results in that frequent words are not as neutral as words from the reviews themselves. There are still product names especially in the negative summaries but overall, models trained on the summaries may be able to generalize a little better than models trained on the reviews.

In [ ]:
'''
Merge datasets in the data folder.
Returns a dataframe of all the data.
'''
def retrieve_reviews_df():
    all_files = glob.glob("./data/*.csv")

    df_list = []

    for filename in all_files:
        print(f"Concatenating {filename}")
        df = pd.read_csv(filename, index_col=None, header=0)
        df_list.append(df)
    
    return pd.concat(df_list, axis=0, ignore_index=True)

In [ ]:
reviews_df = retrieve_reviews_df()
reviews_df.head()

In [ ]:
# number of null values per column
reviews_df.isnull().sum()

In [ ]:
# preprocessing for the dataframe 
def score(x):
    if x<3:
        return -1
    elif x==3:
        return 0
    else:
        return 1

def preprocessing(df):
    # drop columns
    new_df = df.drop(columns = ["ProductId", "UserId", "ProfileName", "HelpfulnessNumerator", "HelpfulnessDenominator", "Time"])

    # drop na values
    new_df = new_df.dropna(axis=0)

    # make a new column of sentiment: (-1/0/1) -- pos/neutral/neg -- 1,2/3/4,5
    new_df['Sentiment'] = new_df.apply(lambda x: score(x['Score']), axis=1)


    return new_df

In [ ]:
df = preprocessing(reviews_df)
df.head()

# Proposed Solution

Before feeding the data to any of our models, we will preprocess and vectorize our dataset using scikit-learn's CountVectorizer and TfidfVectorizer classes.

**Support Vector Machine**: SVM is a supervised learning algorithm that can be used for both classification and regression. The algorithm is based on identifying a hyperplane that best separates the data into classes and maximizes the separation boundaries between the data points.<a name="five"></a>[<sup>[5]</sup>](#five) The hyperplane is constructed by transforming the data with kernels, which can be linear, sigmoid, polynomial, etc. Because SVM in its simplest form supports binary classification, we will be implementing multiclass SVM (either one vs. one, one vs. all, or directed acyclic graph approach). These involve breaking down the classification into multiple binary classification cases. We plan to use scikit-learn's SVM class in our implementation.

**Random Forest Classifier**: RFC is a supervised learning algorithm that can be used for both classification and regression. The algorithm is based on creating several decision trees, which helps in preventing overfitting.<a name="six"></a>[<sup>[6]</sup>](#six) All these trees, when acting together, outperforms any individual tree, a reason we chose this over just a decision tree. When performing classification, it picks the class chosen by the most trees. When performing regression, it takes the averages of all the trees. We plan to use scikit-learn's RandomForestClassifier class in our implementation.

**Logistic Regression**: Logistic Regression is a supervised learning algorithm that can be used for classification.<a name="seven"></a>[<sup>[7]</sup>](#seven) While the name is logistic regression, we are planning to use the multinomial version of logistic regression since we have 3 outcomes for the labels(-1, 0, 1) instead of only 2, which is required for regular logistic regression. Since it is only a classifier, the ratings will be used as a categorical variable rather than a continuous variable just for this model. We plan to use scikit-learn's LogisticRegression class in our implementation.

**Ordinal Regression**: Ordinal Regression is a supervised learning algorithm with both classification and metric regression properties <a name="eight"></a>[<sup>[8]</sup>](#eight) used to predict ordinal variables or variables whose values are ordered making it an attractive solition for classification problems where the classifiers are ordered such as ours. Because the possible labels our model could predict include negative (-1), neutral (0), and positive (1), they can be considered ordinal as they are dependent in terms of their ordering: negative < neutral < positive. We believe this may improve our model's performance rather than treating the labels as just categorical variables. 

Each of these aforedescribed models will be used twice, once when the classifiers are labels (ie -1, 0, 1 for negative, neutral, and positive) and once when the classifiers are continuous ratings (1-5). This means that each of them will be performing both classification and regression once each. All of the models can be found in the sklearn Python library, which we will be using.<a name="nine"></a>[<sup>[9]</sup>](#nine)

To test the models, we will be initially shuffling the data and splitting it into training, validation, and test sets with a split ratio of 60-20-20. We will train the models using only the training set, and we will use the validation set as an intermediate dataset for evaluating the model’s performance as we tweak parameters. Finally we can use the test set to evaluate each of the model’s ability to generalize and accurately predict on unseen data in accordance with our evaluation metrics.

# Evaluation Metrics

For the first three models – the SVM, Random Forest, and Logistic Regression models used to classify the reviews as negative, neutral, or positive –  we will be using classification accuracy (number of correct predictions divided by the total number of predictions made) to analyze and compare to identify the best model.

For the next set of three models –  the SVM, Random Forest, and Logistic Regression used to predict a rating for the reviews on a continuous range between 1 and 5 stars – we will be using root mean squared error (RMSE) to compare the models. The formula for root mean squared error is given by

$$RMSE = \sqrt{\frac{\sum_{i=1}^N (x_i - \hat{x_i})^2}{N}}$$

where $i$ is the index of a datapoint, $N$ is the sample size of the data that is being predicted, $x_i$ is the actual value, and $\hat{x_i}$ is the predicted value.

Because we are using two different metrics to evaluate the classification models and the regression models, we cannot directly compare the performance of the two types of models.

# Preliminary results

More code and detailed outputs can be found in our models directory. In this section, we summarize the values for corresponding metrics that we have obtained so far.


**Logistic Regression**

We ran logistic regression on the processed and vectorized dataset, specifically the "Reviews" column to establish our baseline score. We were able to obtain 80.41359% accuracy.

In [ ]:
clf_logReg_pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), 
                            ("clf_logReg", LogisticRegression(multi_class='multinomial',solver ='newton-cg'))])
                            
clf_logReg_pipe.fit(X_train, y_train)
predictedLogReg = clf_logReg_pipe.predict(X_test)
np.mean(predictedLogReg == y_test)

**Random Forest Classifier**

We also performed random forest classification on the processed and vectorized data. With this model, we obtained an accuracy of 80.31332%.

In [ ]:
clf_randomForest_pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf_randomForest", RandomForestClassifier(verbose=2))])
clf_randomForest_pipe.fit(X_train, y_train)
predictedRandomForest = clf_randomForest_pipe.predict(X_test)
np.mean(predictedRandomForest == y_test)

**Support Vector Machine**

Linear support vector classification resulted in 82.10158% accuracy.

In [ ]:
clf_linearSVC_pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf_linearSVC", LinearSVC(verbose=True))])
clf_linearSVC_pipe.fit(X_train, y_train)

Based on these preliminary results from the models, it appears that classifying the data through SVMs results in the highest accuracy.

# Ethics & Privacy

Some potential concerns with data privacy is that in the data itself there are user IDs and usernames that are easily associated with Amazon accounts. In order to address the issue of the data being attached to user IDs and usernames, we will be dropping those columns as they are also unnecessary to our analysis. The dataset also has a column meant to id each review, so we can use that column to identify a given review.

# Team Expectations

We will be dividing the work in an as equal manner as possible.
- *Expectation 1*: Communicate actively with team members in our Discord group.
- *Expectation 2*: Discuss with the team before making big implementation decisions or changes.
- *Expectation 3*: Meet at least once a week to complete our task for the week as presented by the project timeline proposal.
- *Expectation 4*: Actively participate and contribute to the project.
- *Expectation 5*: Meet the day of or the day before a deadline in order to ensure everything is complete to our standards.

# Project Timeline Proposal

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 4/22  | 1 PM |  Brainstorm topics/questions (all)  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 4/24  | 10:00 PM |  Do background research on topic | Edit, finalize, and submit project proposal | 
| 5/11  | 7:45 PM | Looking at dataset | Downloading dataset, cleaning dataset, and beginning some preliminary exploration |
| 5/12  | 4 PM | Reviewing previous meeting results | Perform EDA, prepare data for use, wrote some further EDA ideas |
| 5/16  | 6 PM | Import & Wrangle Data; Do some EDA | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 5/22  | 12 PM | Finalize wrangling/EDA; Begin programming for project | Discuss/edit project code; Complete project |
| 5/29  | 12 PM | Complete analysis; Draft results/conclusion/discussion | Discuss/edit full project |
| 6/8  | Before 11:59 PM  | NA | Turn in Final Project  |

# Footnotes

<a name="one"></a>1.[^](#one): Introduction to sentiment analysis: What is sentiment analysis? DataRobot AI Cloud. (2022, March 24). Retrieved April 24, 2022, from https://www.datarobot.com/blog/introduction-to-sentiment-analysis-what-is-sentiment-analysis/<br> 
<a name="two"></a>2.[^](#two): read, A. min, Finn Bartram·March 5, 2021·238 views, & views, 9 min read·15381. (2022, April 11). Is sentiment analysis machine learning? The CX Lead. Retrieved April 24, 2022, from https://thecxlead.com/topics/is-sentiment-analysis-machine-learnin g/
<a name="three"></a>3.[^](#three): Sentiment Analysis & Machine Learning. MonkeyLearn Blog. (2020, April 20). Retrieved April 24, 2022, from https://monkeylearn.com/blog/sentiment-analysis-machine-learning/
 <br> <a name="four"></a>4.[^](#four): Sentiment analysis. Papers With Code. (n.d.). Retrieved April 24, 2022, from https://paperswithcode.com/task/sentiment-analysis
 <br> 
<a name="five"></a>5.[^](#five): Reddy, V. (2020, June 23). Sentiment analysis using SVM. Medium. Retrieved April 24, 2022, from https://medium.com/@vasista/sentiment-analysis-using-svm-338d418e3ff1<br>
<a name="six"></a>6.[^](#six): Yiu, T. (2021, September 29). Understanding random forest. Medium. Retrieved April 24, 2022, from https://towardsdatascience.com/understanding-random-forest-58381e0602d2<br>
<a name="seven"></a>7.[^](#seven): Swaminathan, S. (2019, January 18). Logistic regression - detailed overview. Medium. Retrieved April 24, 2022, from https://towardsdatascience.com/logistic-regression-detailed-overview-46c4da4303 <br>
<a name="eight"></a>8.[^](#eight): S. E. Saad and J. Yang, "Twitter Sentiment Analysis Based on Ordinal Regression," in IEEE Access, vol. 7, pp. 163677-163685, 2019, doi: 10.1109/ACCESS.2019.2952127. Retrieved May 17, 2022, from https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8894084 <br> 
<a name="nine"></a>9.[^](#nine): Sucky, R. N. (2021, August 28). A complete sentiment analysis project using Python's Scikit-Learn. Medium. Retrieved April 24, 2022, from https://towardsdatascience.com/a-complete-sentiment-analysis-project-using-pythons-scikit-learn-b9ccbb040c2 <br> 